In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from sklearn.feature_selection import SelectFromModel


# Data transformation (months)

In [2]:
def data_trans_months(data_loc):

    data = pd.read_csv(data_loc)


    features = ['batch size', 'nb of months used', 'prod_mnth','prod_year', 'fail_mnth', 'fail_year', 'percent failures']
    new_data = pd.DataFrame(columns = features)


    labels = ['1. Ay', '2. Ay', '3. Ay', '4. Ay',
       '5. Ay', '6. Ay', '7. Ay', '8. Ay', '9. Ay', '10. Ay', '11. Ay',
       '12. Ay', '13. Ay', '14. Ay', '15. Ay', '16. Ay', '17. Ay', '18. Ay',
       '19. Ay', '20. Ay', '21. Ay', '22. Ay', '23. Ay', '24. Ay', '25. Ay',
       '26. Ay', '27. Ay', '28. Ay', '29. Ay', '30. Ay', '31. Ay', '32. Ay',
       '33. Ay', '34. Ay', '35. Ay', '36. Ay', '37. Ay', '38. Ay', '39. Ay',
       '40. Ay']
    
    i = 0

    for k in labels:
        tmp_data = pd.DataFrame(columns = features)
    
        tmp_data['percent failures'] = data[k]
        tmp_data['batch size'] = data['Üretim Adet']
        tmp_data['nb of months used'] = i + 1
    
        prod_date = pd.to_datetime(data['Üretim Dönemi'])
    
    
        months = list(range(1,13))
    
        while True:
            tmp_data['prod_mnth'] = prod_date.apply(lambda x: months[x.month-1])
            tmp_data['prod_year'] = prod_date.apply(lambda x: x.year)
            break
    
        tmp_data['fail_mnth'] = tmp_data['prod_mnth'] + tmp_data['nb of months used'] % 12    
        tmp_data['fail_year'] = tmp_data['prod_year'] + tmp_data['nb of months used'] // 12
    
    
        if i == 0:
            new_data = tmp_data
        else:
            new_data = new_data.append(tmp_data)
        i += 1

    new_data = pd.concat([new_data, pd.get_dummies(new_data['prod_mnth'], prefix ='prod_mnth')], axis = 1)
    new_data = pd.concat([new_data, pd.get_dummies(new_data['prod_year'], prefix ='prod_year')], axis = 1)
    new_data = pd.concat([new_data, pd.get_dummies(new_data['fail_mnth'], prefix ='fail_mnth')], axis = 1)
    new_data = pd.concat([new_data, pd.get_dummies(new_data['fail_year'], prefix ='fail_year')], axis = 1)



    new_data.drop(['prod_mnth'], axis = 1 , inplace = True)
    new_data.drop(['fail_mnth'], axis = 1 , inplace = True)
    new_data.drop(['prod_year'], axis = 1 , inplace = True)
    new_data.drop(['fail_year'], axis = 1 , inplace = True)



    new_data = new_data.dropna()
    new_data.index = range(len(new_data))
    
    return new_data


# Data transformation (seasons)

In [3]:
#Load our initial dataset to transform
def data_creation(file):
    data = pd.read_csv(file,decimal = ',')
    
    import dateutil.relativedelta

    labels = ['1. Ay', '2. Ay', '3. Ay', '4. Ay',
       '5. Ay', '6. Ay', '7. Ay', '8. Ay', '9. Ay', '10. Ay', '11. Ay',
       '12. Ay', '13. Ay', '14. Ay', '15. Ay', '16. Ay', '17. Ay', '18. Ay',
       '19. Ay', '20. Ay', '21. Ay', '22. Ay', '23. Ay', '24. Ay', '25. Ay',
       '26. Ay', '27. Ay', '28. Ay', '29. Ay', '30. Ay', '31. Ay', '32. Ay',
       '33. Ay', '34. Ay', '35. Ay', '36. Ay', '37. Ay', '38. Ay', '39. Ay',
       '40. Ay']
    
    i = 0
    
    #the new data we'll create
    new_data = pd.DataFrame()


    for l1 in labels:

        labels2 = ['percent failures', 'prod_period','failer_period','prod_season','fail_season','batch size']

        #create a temporary dataframe
        tmp_data = pd.DataFrame(columns = labels2)
        
        tmp_data['percent failures'] = data[l1]
        tmp_data['prod_period'] = data['Üretim Dönemi']
        tmp_data['failer_period'] = i + 1
        tmp_data['prod_period'] = pd.to_datetime(tmp_data['prod_period'])
        tmp_data['batch size'] = data['Üretim Adet']

        #Fill the month_since_prod column
        for j in range(len(tmp_data['prod_period'])):
            dt = tmp_data.iloc[j,1]
            delta = dateutil.relativedelta.relativedelta(months = i + 1)
            tmp_data.iloc[j,2] = delta + dt
            tmp_data.iloc[j,2] = pd.to_datetime(tmp_data.iloc[j,2])


        for j in range(len(tmp_data['prod_season'])):
            tmp_data.iloc[j,1] = tmp_data.iloc[j,1].to_pydatetime()
            if (tmp_data.iloc[j,1].month == 12) or (tmp_data.iloc[j,1].month == 1) or (tmp_data.iloc[j,1].month == 2):
                tmp_data.iloc[j,3] = 'Winter'
            if (tmp_data.iloc[j,1].month == 9) or (tmp_data.iloc[j,1].month == 10) or (tmp_data.iloc[j,1].month == 11):
                tmp_data.iloc[j,3] = 'Autumn'
            if (tmp_data.iloc[j,1].month == 3) or (tmp_data.iloc[j,1].month == 4) or (tmp_data.iloc[j,1].month == 5):
                tmp_data.iloc[j,3] = 'Spring'
            if (tmp_data.iloc[j,1].month == 6) or (tmp_data.iloc[j,1].month == 7) or (tmp_data.iloc[j,1].month == 8):
                tmp_data.iloc[j,3] = 'Summer'

                
        for j in range(len(tmp_data['fail_season'])):
            tmp_data.iloc[j,2] = tmp_data.iloc[j,2].to_pydatetime()
            if (tmp_data.iloc[j,2].month == 12) or (tmp_data.iloc[j,1].month == 1) or (tmp_data.iloc[j,1].month == 2):
                tmp_data.iloc[j,4] = 'Winter'
            if (tmp_data.iloc[j,2].month == 9) or (tmp_data.iloc[j,1].month == 10) or (tmp_data.iloc[j,1].month == 11):
                tmp_data.iloc[j,4] = 'Autumn'
            if (tmp_data.iloc[j,2].month == 3) or (tmp_data.iloc[j,1].month == 4) or (tmp_data.iloc[j,1].month == 5):
                tmp_data.iloc[j,4] = 'Spring'
            if (tmp_data.iloc[j,2].month == 6) or (tmp_data.iloc[j,1].month == 7) or (tmp_data.iloc[j,1].month == 8):
                tmp_data.iloc[j,4] = 'Summer'
                
                
        #Encode the timestamp variables
        for j in range(len(tmp_data['prod_period'])):
            tmp_data.iloc[j,1] = 10000*tmp_data.iloc[j,1].year + 100*tmp_data.iloc[j,1].month + tmp_data.iloc[j,1].day

        for j in range(len(tmp_data['failer_period'])):
            tmp_data.iloc[j,2] = 10000*tmp_data.iloc[j,2].year + 100*tmp_data.iloc[j,2].month + tmp_data.iloc[j,2].day

        i += 1


        #Construct the new data
        new_data = new_data.append(tmp_data)
        #Drop the na values
        new_data = new_data.dropna()

    #One hot encoding for the season column and concatenate it with the original data
    new_data = pd.concat([new_data,pd.get_dummies(new_data['prod_season'], prefix='prod_season')],axis=1)
    new_data = pd.concat([new_data,pd.get_dummies(new_data['fail_season'], prefix='fail_season')],axis=1)
    
    
    #Drop the original season column from the data
    new_data.drop(['prod_season'],axis=1, inplace=True)
    new_data.drop(['fail_season'],axis=1, inplace=True)
    
    new_data.index = range(len(new_data))

    return new_data


# Ibti's random_forest func

In [4]:
def random_forest(data):
    
    
    train_data, test_data = np.split(data, [int(.7 * len(data))])
    
    features = data.columns

    train_y = train_data['percent failures']
    train_x = train_data.drop('percent failures', axis=1)
    test_y = test_data['percent failures']
    test_x = test_data.drop('percent failures',axis=1)
    
    
    
    
    #Random forest with no features selection:
    from sklearn.ensemble import RandomForestRegressor
    random_forest_normal = RandomForestRegressor()
    random_forest_normal.fit(train_x, train_y)
    


    #Predict:
    predictions = random_forest_normal.predict(test_x)
    test_y = np.array(pd.to_numeric(test_y))


    # Compute mean absolute percentage error (MAPE), Accuracy and RMSE
    print("--------------Random Forest with no features selection----------------")
    mape = np.mean(100 * (abs(predictions - test_y)/test_y))
    print('MAPE:',mape)

    accuracy = 100 - mape
    print('Accuracy:', accuracy, '%.')

    rmse = np.sqrt((abs(predictions - test_y) ** 2). mean())
    print('RMSE: ', rmse)
    
    rmspe = np.sqrt((((predictions - test_y) / test_y) ** 2).mean())
    print('RMSPE:',rmspe * 100, '%.')
    
    error_in_ref = sum(round(abs(predictions - test_y) * test_x['batch size']))
    total_ref = sum(test_x['batch size'])
    print('Error in refrigerators: ', error_in_ref, 'misclassified out of: ', total_ref, 'refrigerators.')
    print('Ratio: {} %'. format(error_in_ref * 100 / total_ref))
   
    
    
    print("-------------------------------------------------------------------")



    #Random forest with features selection
    from sklearn.ensemble import RandomForestRegressor
    random_forest = SelectFromModel(RandomForestRegressor())
    random_forest.fit(train_x, train_y)
    
    
    
    #Transform our datasets according to the selected features
    important_train_x = random_forest.transform(train_x)
    important_test_x = random_forest.transform(test_x)
    
    
    # Apply random forest using the new dataset with selected features
    random_forest_important = RandomForestRegressor()

    # Train the new model
    random_forest_important.fit(important_train_x, train_y)
    


    #Predict:
    predictions_important = random_forest_important.predict(important_test_x)
    test_y = np.array(pd.to_numeric(test_y))


    # Compute mean absolute percentage error (MAPE), Accuracy and RMSE
    
    print("--------------Random Forest with features selection-------------------------")
    mape_imp = np.mean(100 * (abs(predictions_important - test_y)/test_y))
    print('MAPE:', mape_imp)

    accuracy_imp = 100 - mape_imp
    print('Accuracy:', accuracy_imp, '%.')
    
    rmse_imp = np.sqrt(((predictions_important - test_y) ** 2). mean())
    print('RMSE: ', rmse_imp)

    rmspe_imp = np.sqrt((((predictions_important - test_y) / test_y) ** 2).mean())
    print('RMSPE:', rmspe_imp * 100, '%.')
    
    
    error_in_ref_imp = sum(round(abs(predictions_important - test_y) * test_x['batch size']))
    total_ref_imp = sum(test_x['batch size'])
    print('Error in refrigerators: ', error_in_ref_imp, 'misclassified out of: ', total_ref_imp, 'refrigerators')
    print('Ratio: {} %'. format(error_in_ref_imp * 100 / total_ref_imp))    

    #num_failed_ref = pd.DataFrame(columns=['failures percentage'])
    #num_failed_ref['failures percentage'] = failures_data['percentage_failure'] * (len(test_data) * (rmse**2)+ 1 )
    #error = abs(num_failed_ref['failures percentage'] - failures_data['percentage_failure'])

    #print('num of refri error:',np.mean(error), 'out of ',sum(test_ref['uretim Adet']))



# RF with valid-test sets and shuffled

In [5]:
def rf_valid_test(new_data):
        
    #shuffle the dataset
    new_data = new_data.reindex(np.random.permutation(new_data.index))
    
    #Split the data: The first 70% of the data for training, 10% for validation and the remaining 20% for test
    train_data, val_data, test_data = np.split(new_data, [int(.7 * len(new_data)), int(.8 * len(new_data))])

    train_y = train_data['percent failures']
    train_x = train_data.drop('percent failures', axis = 1)
    test_y = test_data['percent failures']
    test_x = test_data.drop('percent failures',axis = 1)
    val_y = val_data['percent failures']
    val_x = val_data.drop('percent failures',axis = 1)

    
    #Random forest
    from sklearn.ensemble import RandomForestRegressor
    random_forest = RandomForestRegressor()
    random_forest.fit(train_x, train_y)


    #Predict validation:
    val_predictions = random_forest.predict(val_x)
    val_y = np.array(pd.to_numeric(val_y))

    #Validation
    print('Validation Error')
    
    mape_val = (abs(val_predictions - val_y) / val_y).mean()
    print('MAPE: ', mape_val * 100, '%')

    accuracy_val = 100 * (1 - mape_val)
    print('Accuracy: ', accuracy_val, '%')    
    
    rmse_val = np.sqrt((abs(val_predictions - val_y) ** 2).mean())
    print('RMSE: ', rmse_val)

    rmspe_val = np.sqrt(((abs(val_predictions - val_y) / val_y) ** 2).mean())
    print('RMSPE: ', rmspe_val * 100, '%')


    error_in_ref_val = sum(round(abs(val_predictions - val_y) * val_x['batch size']))
    total_ref_val = sum(val_x['batch size'])
    print('Error in refrigerators: ', error_in_ref_val, 'misclassified out of: ', total_ref_val, 'refrigerators.' )
    print('Ratio: {} %'. format(error_in_ref_val * 100 / total_ref_val))


    print('---------------------------------------------------------------------')

    #Predict test:
    test_predictions = random_forest.predict(test_x)
    test_y = np.array(pd.to_numeric(test_y))


    #Test
    print('Test Error')
    
    mape = (abs(test_predictions - test_y) / test_y).mean()
    print('MAPE: ', mape * 100, '%')    
    
    accuracy = 100 * (1 - mape)
    print('Accuracy: ', accuracy, '%')

    rmse = np.sqrt((abs(test_predictions - test_y) ** 2).mean())
    print('RMSE: ', rmse)

    rmspe = np.sqrt(((abs(test_predictions - test_y) / test_y) ** 2).mean())
    print('RMSPE: ', rmspe * 100, '%')


    error_in_ref = sum(round(abs(test_predictions - test_y) * test_x['batch size']))
    total_ref = sum(test_x['batch size'])
    print('Error in refrigerators: ', error_in_ref, 'misclassified out of: ', total_ref, 'refrigerators.')
    print('Ratio: {} %'. format(error_in_ref * 100 / total_ref))


# SVM Model

In [6]:
def svm_model(data):
    from sklearn import svm
    from sklearn.feature_selection import SelectFromModel
    
   
    #shuffle the dataset
    
    train_data, test_data = np.split(data, [int(.7 * len(data))])
    

    train_y = train_data['percent failures']
    train_x = train_data.drop('percent failures', axis=1)
    test_y = test_data['percent failures']
    test_x = test_data.drop('percent failures',axis=1)
    
    
    #SVM with no selected features
    print("-----------------SVR with no features selection---------------")
    
    svm_mod = svm.SVR(C = 10, epsilon = 0.1, gamma = 1e-07, kernel = 'rbf')
    svm_fit = svm_mod.fit(train_x, train_y)
    
    #Predict:
    svm_predictions = svm_mod.predict(test_x)
    test_y = np.array(pd.to_numeric(test_y))
    
    # Compute mean absolute percentage error (MAPE), Accuracy and RMSE
    mape = np.mean(100 * (abs(svm_predictions - test_y) / test_y))
    print('MAPE:', mape)

    accuracy = 100 - mape
    print('Accuracy:', accuracy, '%.')

    rmspe = np.sqrt((((svm_predictions - test_y) / test_y) ** 2).mean())
    print('RMSPE:', rmspe * 100, '%.')
    
    
    print("-----------------SVR with features selection---------------")
  
    #I couldn't find a way to apply feature selection for svm so I'll try to combine random forest and svr here.
    #I'll find the important features using random forest and apply them in svr.
    
    #Random forest with features selection
    from sklearn.ensemble import RandomForestRegressor
    random_forest = SelectFromModel(RandomForestRegressor())
    random_forest.fit(train_x, train_y)
    
    
    
    #Transform our datasets according to the selected features
    important_train_x = random_forest.transform(train_x)
    important_test_x = random_forest.transform(test_x)  

    #fit the model 
    svm_model_imp = svm.SVR(C= 10, epsilon=0.1, gamma = 1e-07, kernel= 'rbf')
    svm_model_imp.fit(important_train_x, train_y)
    
    
    #Predict
    svm_predict_imp = svm_model_imp.predict(important_test_x)
    test_y = np.array(pd.to_numeric(test_y))
    
    # Compute mean absolute percentage error (MAPE), Accuracy and RMSE
    mape = np.mean(100 * (abs(svm_predict_imp - test_y) / test_y))
    print('MAPE:', mape)

    accuracy = 100 - mape
    print('Accuracy:', accuracy, '%.')

    rmspe = np.sqrt((((svm_predict_imp - test_y) / test_y) ** 2).mean())
    print('RMSPE:', rmspe * 100, '%.')
    
    
    #num_failed_ref = pd.DataFrame(columns=['failures percentage'])
    #num_failed_ref['failures percentage'] = failures_data['percentage_failure'] * (len(test_data) * (rmse**2)+ 1 )
    #error = abs(num_failed_ref['failures percentage'] - failures_data['percentage_failure'])
    
    #print('num of refri error:',np.mean(error), 'out of ',sum(test_ref['uretim Adet']))



# Results

In [9]:
cum_data = r'C:\Users\cigdem\Desktop\pure\40 ay\cum_cum_40.csv'
marg_cum_data = r'C:\Users\cigdem\Desktop\pure\40 ay\marg_cum_40.csv'

datasets = [cum_data, marg_cum_data]
names = ['CUM / CUM Data', 'MAG / CUM Data']

i = 0

for file_loc in datasets:
    
    print(names[i])
    data = data_trans_months(file_loc)
    
    print('**********************************************************************')
    print('\\\\\\\\\\\\\ With one-hot encoded months and years /////////////')
    print('--------------------RF without shuffle, only test--------------------')

    random_forest(data)

    print('-------------RF with shuffled data, valid-test sets---------------')

    rf_valid_test(data)
    
    print('------------------------- SVM Model -------------------------')
    
    svm_model(data)
    

    print('\\\\\\\\\\\\\\\\ With one-hot encoded seasons ////////////////')

    data_season = data_creation(file_loc)

    print('----------------------------------------------------------------------')

    print('--------------------RF without shuffle, only test--------------------')

    random_forest(data_season)

    print('-----------------RF with shuffled data, valid-test sets-------------------')

    rf_valid_test(data_season)
    
    print('------------------------- SVM Model -------------------------')
    
    svm_model(data)
    
    print('**********************************************************************')
    
    i += 1

CUM / CUM Data
**********************************************************************
\\\\\\\ With one-hot encoded months and years /////////////
--------------------RF without shuffle, only test--------------------
--------------Random Forest with no features selection----------------
MAPE: 33.54674233194257
Accuracy: 66.45325766805743 %.
RMSE:  0.056032725729876755
RMSPE: 37.69958992257523 %.
Error in refrigerators:  1248016.0 misclassified out of:  27635833 refrigerators.
Ratio: 4.515934077326347 %
-------------------------------------------------------------------
--------------Random Forest with features selection-------------------------
MAPE: 29.551373508585282
Accuracy: 70.44862649141471 %.
RMSE:  0.05069870745400859
RMSPE: 33.638500150513345 %.
Error in refrigerators:  1114071.0 misclassified out of:  27635833 refrigerators
Ratio: 4.031255363281432 %
-------------RF with shuffled data, valid-test sets---------------
Validation Error
MAPE:  13.186245510414206 %
Accuracy:  86.81

In [10]:
#Here I tried to see whether shuffling has an effect in models that do not use shuffled datasets and it seems like it has an effect.
#I'm gonna leave it in comment for you to check it out.

data = data_creation(r'C:\Users\cigdem\Desktop\pure\40 ay\cum_cum_40.csv')
data = data.reindex(np.random.permutation(data.index))
random_forest(data)

--------------Random Forest with no features selection----------------
MAPE: 13.468733856105068
Accuracy: 86.53126614389492 %.
RMSE:  0.005412370844207806
RMSPE: 39.81348417987172 %.
Error in refrigerators:  80600.0 misclassified out of:  21409500 refrigerators.
Ratio: 0.3764683902006119 %
-------------------------------------------------------------------
--------------Random Forest with features selection-------------------------
MAPE: 12.69726206402096
Accuracy: 87.30273793597904 %.
RMSE:  0.006250275218609698
RMSPE: 38.63530879203552 %.
Error in refrigerators:  84042.0 misclassified out of:  21409500 refrigerators
Ratio: 0.3925453653751839 %


In [9]:
#data = data_creation(r'C:\Users\cigdem\Desktop\pure\40 ay\marg_cum_40.csv')
#pd.DataFrame.to_excel(data, r'C:\Users\cigdem\Desktop\pure\40 ay\encodetwo.xlsx')
